# 📊 SLA Analysis Dashboard

**Purpose**: Track SLA compliance and identify work packages past their requested delivery date.

## Key Metrics
- SLA Breach % (orders past requested delivery date)
- Days Overdue analysis
- Breach % by Product
- At-Risk Orders (approaching SLA deadline)

---

## 1. Setup & Data Loading

In [ ]:
# Install required packages
!pip install pandas openpyxl plotly seaborn matplotlib -q

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Libraries loaded successfully!")

In [ ]:
# File path - update this to match your file location
filename = r"C:\Users\bmalaraju\Documents\WP-OP Agent\JIRA-Agent\11.25.WP Orders_25-11-2025_v01.xlsx"
print(f"📁 Using file: {filename}")

In [ ]:
# Load and prepare data
df = pd.read_excel(filename, engine='openpyxl')

# Display basic info
print(f"📊 Dataset loaded: {len(df):,} rows, {len(df.columns)} columns")
print(f"\n📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

# Show column names
print("\n📋 Available columns:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2}. {col}")

## 2. Data Preparation

In [ ]:
# Column mapping - adjust if your columns have different names
COLUMN_MAP = {
    'requested_date': 'WP Requested Delivery Date',
    'readiness_date': 'WP Readiness Date',
    'status': 'WP Order Status',
    'product': 'Product',
    'customer': 'Customer',
    'order_id': 'WP Order ID',
    'wp_id': 'WP ID',
    'wp_name': 'WP Name',
    'quantity': 'WP Quantity',
    'in_time_delivery': 'In-Time Delivery',
    'added_date': 'Added Date',
    'project_name': 'Project Name',
    'domain': 'Domain'
}

# Validate columns exist
missing_cols = [v for v in COLUMN_MAP.values() if v not in df.columns]
if missing_cols:
    print(f"⚠️ Missing columns: {missing_cols}")
    print("\nPlease update COLUMN_MAP with correct column names from your file.")
else:
    print("✅ All required columns found!")

In [ ]:
# Parse dates and prepare analysis dataframe
analysis_df = df.copy()

# Helper to strip timezone from dates
def parse_date(series):
    dt = pd.to_datetime(series, errors='coerce')
    if dt.dt.tz is not None:
        dt = dt.dt.tz_localize(None)
    return dt

# Parse requested delivery date
analysis_df['target_date'] = parse_date(analysis_df[COLUMN_MAP['requested_date']])

# Fallback to readiness date if requested date is missing
if COLUMN_MAP['readiness_date'] in analysis_df.columns:
    readiness = parse_date(analysis_df[COLUMN_MAP['readiness_date']])
    analysis_df['target_date'] = analysis_df['target_date'].fillna(readiness)

# Parse added date for trend analysis
analysis_df['added_date'] = parse_date(analysis_df[COLUMN_MAP['added_date']])

# Current date for SLA calculation
TODAY = pd.Timestamp.now().normalize()

# Terminal statuses (orders that are complete and shouldn't count as breached)
TERMINAL_STATUSES = ['Approved', 'Cancelled', 'Rejected']

# Calculate SLA metrics
analysis_df['is_terminal'] = analysis_df[COLUMN_MAP['status']].isin(TERMINAL_STATUSES)
analysis_df['has_target_date'] = analysis_df['target_date'].notna()
analysis_df['is_past_due'] = (analysis_df['target_date'] < TODAY) & analysis_df['has_target_date']
analysis_df['is_breached'] = analysis_df['is_past_due'] & ~analysis_df['is_terminal']
analysis_df['days_overdue'] = np.where(
    analysis_df['is_past_due'],
    (TODAY - analysis_df['target_date']).dt.days,
    0
)

# At-risk: due within 7 days, not terminal
analysis_df['is_at_risk'] = (
    (analysis_df['target_date'] >= TODAY) & 
    (analysis_df['target_date'] <= TODAY + timedelta(days=7)) &
    ~analysis_df['is_terminal'] &
    analysis_df['has_target_date']
)

print(f"✅ Data prepared successfully!")
print(f"\n📅 Analysis date: {TODAY.strftime('%Y-%m-%d')}")
print(f"📊 Orders with target dates: {analysis_df['has_target_date'].sum():,}")

## 3. Overall SLA Summary

In [ ]:
# Calculate overall metrics
total_orders = len(analysis_df)
orders_with_date = analysis_df['has_target_date'].sum()
active_orders = (~analysis_df['is_terminal'] & analysis_df['has_target_date']).sum()
breached_orders = analysis_df['is_breached'].sum()
at_risk_orders = analysis_df['is_at_risk'].sum()
compliant_active = active_orders - breached_orders - at_risk_orders

# Percentages
breach_pct = (breached_orders / active_orders * 100) if active_orders > 0 else 0
at_risk_pct = (at_risk_orders / active_orders * 100) if active_orders > 0 else 0
compliant_pct = 100 - breach_pct - at_risk_pct

# Display summary
print("="*60)
print("📊 SLA SUMMARY REPORT")
print("="*60)
print(f"\n📋 Total Orders:           {total_orders:,}")
print(f"📅 With Target Date:        {orders_with_date:,}")
print(f"🔄 Active (Non-Terminal):   {active_orders:,}")
print(f"\n🚨 BREACHED (Past SLA):     {breached_orders:,} ({breach_pct:.1f}%)")
print(f"⚠️  AT RISK (Due ≤7 days):  {at_risk_orders:,} ({at_risk_pct:.1f}%)")
print(f"✅ COMPLIANT:               {compliant_active:,} ({compliant_pct:.1f}%)")
print("="*60)

In [ ]:
# SLA Compliance Pie Chart
labels = ['🚨 Breached', '⚠️ At Risk', '✅ Compliant']
values = [breached_orders, at_risk_orders, compliant_active]
colors = ['#FF6B6B', '#FFE66D', '#4ECDC4']

fig = go.Figure(data=[go.Pie(
    labels=labels,
    values=values,
    hole=0.4,
    marker_colors=colors,
    textinfo='percent+value',
    textfont_size=14,
    hovertemplate='%{label}<br>Orders: %{value}<br>Percentage: %{percent}<extra></extra>'
)])

fig.update_layout(
    title={
        'text': 'SLA Compliance Overview',
        'x': 0.5,
        'font': {'size': 20}
    },
    annotations=[{
        'text': f'{compliant_pct:.0f}%<br>Compliant',
        'x': 0.5, 'y': 0.5,
        'font_size': 16,
        'showarrow': False
    }],
    showlegend=True,
    legend={'orientation': 'h', 'y': -0.1},
    height=500
)

fig.show()

In [ ]:
# SLA Compliance Gauge
fig = go.Figure(go.Indicator(
    mode="gauge+number+delta",
    value=compliant_pct,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "SLA Compliance Rate", 'font': {'size': 24}},
    delta={'reference': 85, 'increasing': {'color': "green"}},
    gauge={
        'axis': {'range': [0, 100], 'tickwidth': 1, 'tickcolor': "darkblue"},
        'bar': {'color': "#4ECDC4"},
        'bgcolor': "white",
        'borderwidth': 2,
        'bordercolor': "gray",
        'steps': [
            {'range': [0, 50], 'color': '#FF6B6B'},
            {'range': [50, 75], 'color': '#FFE66D'},
            {'range': [75, 100], 'color': '#C8E6C9'}
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': 85
        }
    }
))

fig.update_layout(height=400)
fig.show()

## 4. SLA Breach Analysis by Product

In [ ]:
# Calculate breach rate by product
product_col = COLUMN_MAP['product']

product_stats = analysis_df[analysis_df['has_target_date'] & ~analysis_df['is_terminal']].groupby(product_col).agg(
    total_active=('is_breached', 'count'),
    breached=('is_breached', 'sum'),
    at_risk=('is_at_risk', 'sum'),
    avg_days_overdue=('days_overdue', lambda x: x[x > 0].mean() if (x > 0).any() else 0)
).reset_index()

product_stats['breach_pct'] = (product_stats['breached'] / product_stats['total_active'] * 100).round(1)
product_stats['at_risk_pct'] = (product_stats['at_risk'] / product_stats['total_active'] * 100).round(1)
product_stats = product_stats.sort_values('breach_pct', ascending=False)

# Display top products by breach rate
print("\n🚨 SLA BREACH % BY PRODUCT (Top 15)")
print("="*80)
display_df = product_stats.head(15)[[
    product_col, 'total_active', 'breached', 'breach_pct', 'at_risk', 'avg_days_overdue'
]].copy()
display_df.columns = ['Product', 'Active Orders', 'Breached', 'Breach %', 'At Risk', 'Avg Days Overdue']
display_df['Avg Days Overdue'] = display_df['Avg Days Overdue'].round(0).astype(int)
print(display_df.to_string(index=False))

In [ ]:
# Bar chart: Breach % by Product
top_products = product_stats.head(15)

fig = go.Figure()

fig.add_trace(go.Bar(
    y=top_products[product_col],
    x=top_products['breach_pct'],
    orientation='h',
    name='Breach %',
    marker_color='#FF6B6B',
    text=top_products['breach_pct'].apply(lambda x: f'{x:.1f}%'),
    textposition='outside',
    hovertemplate='%{y}<br>Breach Rate: %{x:.1f}%<extra></extra>'
))

fig.update_layout(
    title={'text': 'SLA Breach Rate by Product', 'x': 0.5, 'font': {'size': 20}},
    xaxis_title='Breach %',
    yaxis_title='Product',
    yaxis={'categoryorder': 'total ascending'},
    height=600,
    margin={'l': 200}
)

# Add target line at 15%
fig.add_vline(x=15, line_dash="dash", line_color="green", 
              annotation_text="Target: 15%", annotation_position="top")

fig.show()

In [ ]:
# Stacked bar: Compliant vs Breached vs At-Risk by Product
top_products['compliant'] = top_products['total_active'] - top_products['breached'] - top_products['at_risk']

fig = go.Figure()

fig.add_trace(go.Bar(
    y=top_products[product_col],
    x=top_products['compliant'],
    name='✅ Compliant',
    orientation='h',
    marker_color='#4ECDC4'
))

fig.add_trace(go.Bar(
    y=top_products[product_col],
    x=top_products['at_risk'],
    name='⚠️ At Risk',
    orientation='h',
    marker_color='#FFE66D'
))

fig.add_trace(go.Bar(
    y=top_products[product_col],
    x=top_products['breached'],
    name='🚨 Breached',
    orientation='h',
    marker_color='#FF6B6B'
))

fig.update_layout(
    title={'text': 'SLA Status Distribution by Product', 'x': 0.5, 'font': {'size': 20}},
    barmode='stack',
    xaxis_title='Number of Orders',
    yaxis_title='Product',
    yaxis={'categoryorder': 'total ascending'},
    height=600,
    margin={'l': 200},
    legend={'orientation': 'h', 'y': 1.1}
)

fig.show()

## 5. SLA Analysis by Customer

In [ ]:
# Heatmap: Breach count by Product x Customer
customer_col = COLUMN_MAP['customer']

# Get top products and customers by breach count
top_n = 10
top_breach_products = product_stats.head(top_n)[product_col].tolist()

customer_stats = analysis_df[analysis_df['is_breached']].groupby(customer_col).size()
top_customers = customer_stats.nlargest(top_n).index.tolist()

# Filter data for heatmap
heatmap_data = analysis_df[
    analysis_df[product_col].isin(top_breach_products) & 
    analysis_df[customer_col].isin(top_customers) &
    analysis_df['is_breached']
].groupby([product_col, customer_col]).size().unstack(fill_value=0)

if not heatmap_data.empty:
    fig = px.imshow(
        heatmap_data,
        labels=dict(x="Customer", y="Product", color="Breach Count"),
        x=heatmap_data.columns,
        y=heatmap_data.index,
        color_continuous_scale='Reds',
        title='SLA Breaches Heatmap: Product × Customer'
    )
    
    fig.update_layout(
        height=500,
        xaxis_tickangle=-45
    )
    
    fig.show()
else:
    print("No breach data available for heatmap visualization.")

## 6. SLA Trends Over Time

In [ ]:
# Weekly SLA breach trend
trend_df = analysis_df[analysis_df['added_date'].notna()].copy()
trend_df['week'] = trend_df['added_date'].dt.to_period('W').dt.start_time

weekly_stats = trend_df.groupby('week').agg(
    total=('is_breached', 'count'),
    breached=('is_breached', 'sum')
).reset_index()

weekly_stats['breach_pct'] = (weekly_stats['breached'] / weekly_stats['total'] * 100).round(1)

# Filter to recent 12 weeks
weekly_stats = weekly_stats.tail(12)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=weekly_stats['week'], y=weekly_stats['breached'], name='Breached Orders',
           marker_color='#FF6B6B'),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=weekly_stats['week'], y=weekly_stats['breach_pct'], name='Breach %',
               mode='lines+markers', line=dict(color='#2C3E50', width=3)),
    secondary_y=True
)

fig.update_layout(
    title={'text': 'Weekly SLA Breach Trend', 'x': 0.5, 'font': {'size': 20}},
    xaxis_title='Week',
    height=450,
    legend={'orientation': 'h', 'y': 1.1}
)

fig.update_yaxes(title_text="Breached Orders", secondary_y=False)
fig.update_yaxes(title_text="Breach %", secondary_y=True)

fig.show()

## 7. Days Overdue Analysis

In [ ]:
# Days overdue distribution
overdue_df = analysis_df[analysis_df['is_breached'] & (analysis_df['days_overdue'] > 0)].copy()

if len(overdue_df) > 0:
    # Bucket the days overdue
    bins = [0, 7, 14, 30, 60, 90, float('inf')]
    labels = ['1-7 days', '8-14 days', '15-30 days', '31-60 days', '61-90 days', '90+ days']
    overdue_df['overdue_bucket'] = pd.cut(overdue_df['days_overdue'], bins=bins, labels=labels)
    
    bucket_counts = overdue_df['overdue_bucket'].value_counts().sort_index()
    
    colors = ['#FFE66D', '#FFD93D', '#FF9F1C', '#FF6B35', '#FF4444', '#8B0000']
    
    fig = go.Figure(data=[go.Bar(
        x=bucket_counts.index.astype(str),
        y=bucket_counts.values,
        marker_color=colors,
        text=bucket_counts.values,
        textposition='outside'
    )])
    
    fig.update_layout(
        title={'text': 'Distribution of Days Overdue', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='Days Overdue Category',
        yaxis_title='Number of Orders',
        height=400
    )
    
    fig.show()
    
    # Summary stats
    print("\n📊 Days Overdue Statistics:")
    print(f"   Mean:   {overdue_df['days_overdue'].mean():.1f} days")
    print(f"   Median: {overdue_df['days_overdue'].median():.1f} days")
    print(f"   Max:    {overdue_df['days_overdue'].max():.0f} days")
else:
    print("✅ No overdue orders found!")

## 8. Most Critical Orders (Top 20 Overdue)

In [ ]:
# Top 20 most overdue orders
critical_orders = analysis_df[
    analysis_df['is_breached'] & (analysis_df['days_overdue'] > 0)
].nlargest(20, 'days_overdue')[[
    COLUMN_MAP['order_id'],
    COLUMN_MAP['product'],
    COLUMN_MAP['customer'],
    COLUMN_MAP['status'],
    'target_date',
    'days_overdue'
]].copy()

critical_orders.columns = ['Order ID', 'Product', 'Customer', 'Status', 'Target Date', 'Days Overdue']
critical_orders['Target Date'] = critical_orders['Target Date'].dt.strftime('%Y-%m-%d')

print("\n🚨 TOP 20 MOST CRITICAL OVERDUE ORDERS")
print("="*100)
print(critical_orders.to_string(index=False))

## 9. At-Risk Orders (Due Within 7 Days)

In [ ]:
# At-risk orders list
at_risk_df = analysis_df[analysis_df['is_at_risk']][[
    COLUMN_MAP['order_id'],
    COLUMN_MAP['product'],
    COLUMN_MAP['customer'],
    COLUMN_MAP['status'],
    'target_date'
]].copy()

at_risk_df['days_until_due'] = (at_risk_df['target_date'] - TODAY).dt.days
at_risk_df = at_risk_df.sort_values('days_until_due')
at_risk_df.columns = ['Order ID', 'Product', 'Customer', 'Status', 'Target Date', 'Days Until Due']
at_risk_df['Target Date'] = pd.to_datetime(at_risk_df['Target Date']).dt.strftime('%Y-%m-%d')

print(f"\n⚠️ AT-RISK ORDERS (Due within 7 days): {len(at_risk_df)} orders")
print("="*100)
if len(at_risk_df) > 0:
    print(at_risk_df.head(20).to_string(index=False))
    if len(at_risk_df) > 20:
        print(f"\n... and {len(at_risk_df) - 20} more orders")
else:
    print("✅ No at-risk orders found!")

## 10. Export Results

In [ ]:
# Export summary to Excel
from datetime import datetime

export_filename = f"sla_analysis_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"

with pd.ExcelWriter(export_filename, engine='openpyxl') as writer:
    # Summary sheet
    summary_data = pd.DataFrame({
        'Metric': ['Total Orders', 'Active Orders', 'Breached', 'At Risk', 'Compliant', 
                   'Breach %', 'At Risk %', 'Compliance %'],
        'Value': [total_orders, active_orders, breached_orders, at_risk_orders, compliant_active,
                  f'{breach_pct:.1f}%', f'{at_risk_pct:.1f}%', f'{compliant_pct:.1f}%']
    })
    summary_data.to_excel(writer, sheet_name='Summary', index=False)
    
    # Product breakdown
    product_stats.to_excel(writer, sheet_name='By Product', index=False)
    
    # Critical orders
    critical_orders.to_excel(writer, sheet_name='Critical Orders', index=False)
    
    # At-risk orders
    at_risk_df.to_excel(writer, sheet_name='At Risk Orders', index=False)

print(f"\n✅ Results exported to: {export_filename}")

# Download the file
# files.download() - uncomment if using Colab
# files.download(export_filename)

---

## 📋 Summary

This notebook analyzed SLA compliance across your work package orders:

| Metric | Description |
|--------|-------------|
| **SLA Breach** | Orders past their requested delivery date (non-terminal status) |
| **At Risk** | Orders due within 7 days (non-terminal status) |
| **Compliant** | Active orders with target date in the future |
| **Days Overdue** | Calendar days past the target date |

### Key Actions
1. Review the **Critical Orders** list for immediate attention
2. Focus on **At-Risk Orders** to prevent future breaches
3. Investigate products with high breach rates
4. Use the exported Excel file for detailed follow-up